# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

 **Исследование поведения клиентов сотовой связи** 
 
Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы.  Основная задача проекта заключается в построении модели, которая позволит подобрать новый тариф для старых пользователей.

 **Цель исследования**
 
Цель проекта заключается в анализе данных оператора мобильной связи, а именно:
1. Проанализировать основную информацию о пользователях тарифами.
2. Разбить данные на три части: обучающую, валидационную и тестовую.
3. Провести исследование моделей.
4. Проверить модели на тестовой выборке.

**Ход исследования**

Данные для исследования получены из одного датафрейма. В  данных есть информация о пользователях, а именно сколько времени они тратят на звонки, какое количество СМС отправляют, величина используемоего интернет-траффика, а также о наименовании тарифа,каким он пользовался в течение месяца.

Таким образом, исследование будет состоять из следующих этапов:
- обзор данных;
- предобработка данных (принеобходимости);
- разбиение данных на выборки;
- исследование трех видов моделей (Модель "Решающее дерево", Модель "Случайный лес",Модель "Логистическая регрессия");
- проверка моделей на тестовой выборке;
- проверка модели на адекватность;
- основные выводы.


## Откройте и изучите файл

In [94]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

In [95]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [96]:
df.head(5)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Проверим тип данных.

In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Приведем тип данных в столбцах calls и messages к int64 и проверим данные на явные дубликаты.

In [98]:
for column in df[['calls','messages']]:
    df[column] = df[column].astype("int")  
 

In [99]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   int64  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int64  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 125.7 KB


In [100]:
df.duplicated().sum()

0

In [101]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [102]:
df.shape

(3214, 5)

## Разбейте данные на выборки

### Проведем первоначальную оценку данных с разбиением их по тарифам.

In [103]:
data1 = df[df['is_ultra'] == 1]

In [104]:
data1.describe()

,calls,minutes,messages,mb_used,is_ultra
count,985.000000,985.000000,985.000000,985.000000,985.0
mean,73.392893,511.224569,49.363452,19468.823228,1.0
std,43.916853,308.031100,47.804457,10087.178654,0.0
min,0.000000,0.000000,0.000000,0.000000,1.0
25%,41.000000,276.030000,6.000000,11770.280000,1.0
50%,74.000000,502.550000,38.000000,19308.010000,1.0
75%,104.000000,730.050000,79.000000,26837.720000,1.0
max,244.000000,1632.060000,224.000000,49745.730000,1.0


In [105]:
data0 = df[df['is_ultra'] == 0]

In [106]:
data0.describe()

,calls,minutes,messages,mb_used,is_ultra
count,2229.000000,2229.000000,2229.000000,2229.000000,2229.0
mean,58.463437,405.942952,33.384029,16208.466949,0.0
std,25.939858,184.512604,28.227876,5870.498853,0.0
min,0.000000,0.000000,0.000000,0.000000,0.0
25%,40.000000,274.230000,10.000000,12643.050000,0.0
50%,60.000000,410.560000,28.000000,16506.930000,0.0
75%,76.000000,529.510000,51.000000,20043.060000,0.0
max,198.000000,1390.220000,143.000000,38552.620000,0.0


Данных по тарифу Смарт более чем в 2 раза больше чем данных Ультра. Это не очень хорошо, т.к. при разбиении датасета на выборки мы можем получить, например, очень малое количество данных Ультра в обучающей выборке и модель плохо обучится.

### Разделим исходные данные на обучающую, валидационную и тестовую выборки.

Разделим данные на целевой столбец и признаки.

In [107]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

Выделим набор данных для обучающей, валидационной и тестовой выборки в два этапа.
Отделим сначала обучающую выборку, равной 60% от исходных данных. Зададим параметр random_state чтобы можно было воспроизвести это псевдослучайное разбиение данных.

In [108]:
df_train, df_check = train_test_split(df, test_size=0.4, random_state=12345)


Затем проверочный набор поделим пополам на валидационный набор и тестовый.

In [109]:
df_valid, df_test = train_test_split(df_check, test_size=0.5, random_state=12345)

In [110]:
print(df_train.shape)
print(df_valid.shape)
print(df_test.shape)

(1928, 5)
(643, 5)
(643, 5)


Провели первоначальную оценку данных с разбиением их по тарифам, которая позволила установить, что данных по тарифу Смарт более чем в 2 раза больше чем данных Ультра. Это не очень хорошо, т.к. при разбиении датасета на выборки мы можем получить, например, очень малое количество данных Ультра в обучающей выборке и модель плохо обучится.
Далее данные были разделены на обучающую и проверочную выборки в пропорции 60% на 40%. Затем проверочный набор поделили пополам на валидационный набор и тестовый.

## Исследуйте модели

Рассмотрим наши данные с точки зрения построения прогностической модели.

В качестве целевого признака выступает столбец "is_ultra". Т.к. в нем отражено какой тариф использует клиент, Смарт или Ультра, следовательно этот признак является категориальным. Следовательно нам необходимо решить задачу классификации, а точнее бинарной классификации (т.к. целевых признаков всего два).

Чтобы предложить клиенту новый тариф мы должны предположить как работают взаимосвязи признаков у пользователей этих новых тарифов. На основании этих предположений сможем предсказать какой тариф предложить клиенту. Если эти предсказания соответсвуют реальности, то мы на верном пути и наши предположения верны.
Такой подход называется моделированием, а сами предположения и способы предсказания — моделями машинного обучения.

В процессе моделирования будем использовать следующие модели машинного обучения:

Решающее дерево
Случайный лес
Логистическая регрессия
У "Решающего дерева" и "Случайного леса" есть гиперпараметры, меняя которые можно подобрать наилучшую модель. У "Решающего дерева" это максимальная глубина дерева max_depth. У "Случайного леса" к максимальной глубине добавляется еще один гиперпараметр количество деревьев n_estimators.

Каждую модель обучим на обучающем наборе и проверим на валидационной выборке. Таким образом определим лучшую модель.

Объявим переменные features и target (признаки и целевой признак), которым присвоим соответствующие им признаки.

In [111]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

In [112]:
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

In [113]:
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

### Модель "Решающее дерево"

In [114]:
best_model_tree = None
best_result_tree = 0

for depth in range(1, 15):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth) # обучение модели с заданной глубиной деревьев
    model_tree.fit(features_train, target_train) # обучение модели
    predictions_tree = model_tree.predict(features_valid) # получение предсказания модели
    result_tree = accuracy_score(target_valid, predictions_tree) # определение качества модели
    if result_tree > best_result_tree:
        best_model_tree = model_tree
        best_result_tree = result_tree
        best_depth = depth 
        
print("Accuracy лучшей модели:", best_result_tree)
print("Depth лучшей модели:", best_depth)

Accuracy лучшей модели: 0.7853810264385692
Depth лучшей модели: 3


### Модель "Случайный лес"

In [115]:
best_accuracy_forest = 0
best_depth_forest = 0
for depth in range(1,10):
    for est in range(5,45,5):
        for sample in range(2,5):
            model_forest = RandomForestClassifier(max_depth=depth, n_estimators=est, 
                                           min_samples_leaf=sample, random_state=1515)
            model_forest.fit(features_train, target_train)
            accuracy_forest = model_forest.score(features_valid, target_valid)
            if accuracy_forest > best_accuracy_forest:
                best_accuracy_forest = accuracy_forest
                best_depth_forest = depth
                best_est_forest = est
                best_sample_forest = sample
                best_model_forest = model_forest


print("Accuracy наилучшей модели на валидационной выборке:", best_accuracy_forest)
print("Глубина дерева:", best_depth_forest)
print("N_estimators лучшей модели:", best_est_forest)
print('Мин. объектов в узле:', best_sample_forest)

Accuracy наилучшей модели на валидационной выборке: 0.8055987558320373
Глубина дерева: 7
N_estimators лучшей модели: 25
Мин. объектов в узле: 3


### Модель "Логистическая регрессия"

In [116]:
model_log = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000) #обучим модель
model_log.fit(features_train, target_train) # обучим модель на тренировочной выборке
predictions_log = model_log.predict(features_valid) # получим предсказания модели
result_log = accuracy_score(target_valid, predictions_log)  # посчитаем качество модели для предсказания

print("Accuracy наилучшей модели на валидационной выборке:", result_log)

Accuracy наилучшей модели на валидационной выборке: 0.7107309486780715


Основываясь на показателе точности, можно прийти к выводу, что наиболее качественные результаты прогнозирует модель "случайного леса" с показателем accuracy 0.8. Модель "Решающее дерево" немного уступает в точности, но требует меньшей производительности при выполнении расчетов. Модель "Логистическая регрессия" показала наименьшую точность в 0.71.

## Проверьте модель на тестовой выборке

### Модель "Логистическая регрессия"

In [117]:
model_lr = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000) 
model_lr.fit(features_train, target_train)
accuracy = model_lr.score(features_test, target_test)
print("Качество:", accuracy)

Качество: 0.6842923794712286


In [118]:
predictions_lr_test = model_lr.predict(features_test)
accuracy_lr_test = round(accuracy_score(target_test, predictions_lr_test), 4)
accuracy_lr_test

0.6843

In [119]:
prediction_test_LR = model_log.predict(features_test)
accuracy_LR_test = accuracy_score(prediction_test_LR, target_test)
print(f'Логистическая регрессия: {accuracy_LR_test}')

Логистическая регрессия: 0.6842923794712286


### Модель "Решающее дерево"

In [120]:
model = DecisionTreeClassifier(max_depth=3, random_state=1515)
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Качество:", accuracy)

Качество: 0.7791601866251944


In [121]:
prediction_test_DT = best_model_tree.predict(features_test)
accuracy_DT_test = accuracy_score(prediction_test_DT, target_test)
print(f'Решающее дерево: {accuracy_DT_test}')



Решающее дерево: 0.7791601866251944


### Модель "Случайный лес"

In [122]:
model = RandomForestClassifier(max_depth=7, n_estimators=25, min_samples_leaf=3, random_state=1515)
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Качество:", accuracy)

Качество: 0.7993779160186625


In [123]:
prediction_test_RF = best_model_forest.predict(features_test)
accuracy_RF_test = accuracy_score(prediction_test_RF, target_test)
print(f'Случайный лес: {accuracy_RF_test}')

Случайный лес: 0.7993779160186625


По результатам проверки моделей на тестовой выборке подтвердилось, что модель "Случайного леса" обладает наибольшей точностью. Значения модели "Решающего дерева" близки к показателям модели "Случайного леса". Наименее точной является модель логистической регрессии. Таким образом можно использовать модель "Случайного леса" для выбора тарифа ("Смарт" или "Ультра") и предложения его клиентам заказчика.

## (Бонус)

In [124]:
dmb = DummyClassifier(strategy="stratified" ,random_state=1515)
dmb.fit(features_train, target_train)
acc = dmb.score(features_test, target_test)
print("Качество наивного прогноза :", acc)
print("Качество нашего прогноза :", accuracy)

Качество наивного прогноза : 0.5800933125972006
Качество нашего прогноза : 0.7993779160186625


Качество нашего прогноза гораздо выше.

## Выводы по работе

По результатам исследования была выполнена задача по построению модели для задачи классификации, которая способна подобрать подходящий тариф. Был рассмотрен массив данных о поведении клиентов, которые уже перешли на предлагаемые тарифы. В целом данные чистые, была лишь произведена замена типа данных в calls и messages на int64.
Далее данные были разделены на обучающую и проверочную выборки в пропорции 60% на 40%. Затем проверочный набор поделили пополам на валидационный набор и тестовый.

Данные были рассмотрены с точки зрения построения прогностической модели. В качестве целевого признака выступает столбец "is_ultra". Т.к. в нем отражено какой тариф использует клиент, Смарт или Ультра, следовательно этот признак является категориальным. Необходимо решить задачу бинарной классификации.

Исследованы качество разных моделей и получены следующие результаты:
Дерево решений с точностью 78% и глубиной дерева (max_depth) - 3;
Случайный лес с точностью 80% и количеством деревьев (n_estimators) - 25;
Логистическая регрессия (гиперпараметры по умолчанию) с точностью 71%.
Качество моделей проверено на тестовой выборке и получены следующие результаты:
Дерево решений - точность 78%;
Случайный лес - точность 79%;
Логистическая регрессия - точность 68%.

Соответственно, модель "Случайного леса" может быть использована для подбора тарифа.
Сравнив данную модель со случайными предсказаниями и оценив ее качество на тестовом наборе данных (результат составил 80% правильных ответов) было подтверждено, что данная модель подходит для задачи заказчика. Таким образом можно использовать данную модель для предложения клиентам заказчика нового тарифа ("Смарт" или "Ультра").